In [20]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam


In [21]:
data = pd.read_csv('BS_data.csv')
data

,Company,Time,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt,Price
0,RHI,2013-12-01,1490.3,570.6,919.6,135.3,6.80,718.3,5.31,1.4,-274.3,41.99
1,RCL,2011-12-01,19804.4,11396.6,8407.8,217.5,38.74,7442.4,34.29,8507.2,8245.0,24.77
2,NVDA,2020-01-01,17315.0,5111.0,12204.0,612.0,19.93,11537.0,18.84,2643.0,-8254.0,236.43
3,STC,2012-12-01,1291.2,710.8,580.4,21.1,29.31,263.5,13.58,71.2,-137.4,26.00
4,XRAY,2010-12-01,3258.0,1348.0,1909.9,142.1,12.98,457.6,3.23,786.6,246.6,34.17
5,GWW,2016-12-01,5694.3,3788.5,1905.8,58.8,30.57,684.7,11.64,2247.1,1972.9,232.25
6,SKM,2015-12-01,24288.4,11223.5,13064.9,70.6,183.55,9386.2,132.93,7039.9,5790.9,20.15
7,BAX,2011-12-01,19073.0,12245.0,6828.0,560.3,11.74,3442.0,6.14,5195.0,2290.0,26.88
8,MAR,2012-12-01,6342.0,7627.0,-1285.0,312.3,-4.13,-3274.0,-10.53,2935.0,2847.0,37.27
9,SJM,2019-04-01,16711.3,8740.8,7970.5,113.7,70.08,-5059.2,-44.48,5959.9,5858.6,122.63


In [22]:
prices = data['Price'] 
data = data.drop(['Company', 'Time', 'Price'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price', prices)

# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:9] # Parameters
y = NpMatrix[:,9] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

Total Assets
Done
Total Liabilities
Done
Total Equity
Done
Total Shares Out. on Filing Date
Done
Book Value / Share
Done
Tangible Book Value
Done
Tangible Book Value Per Share
Done
Total Debt
Done
Net Debt
Done
X: [[1.63843386e-03 6.82611560e-04 2.81693214e-02 ... 1.34060792e-03
  2.92521072e-06 1.85568632e-01]
 [2.19349988e-02 1.36816489e-02 4.51864494e-02 ... 1.49174313e-03
  1.77752519e-02 2.00645972e-01]
 [1.91761262e-02 6.13437854e-03 5.38134120e-02 ... 1.41116898e-03
  5.52237995e-03 1.71446269e-01]
 ...
 [6.79067888e-03 4.97604015e-03 3.06086506e-02 ... 1.34050362e-03
  4.01380699e-03 1.89340224e-01]
 [2.35835177e-03 1.36510305e-03 2.83540777e-02 ... 1.34947368e-03
  1.52340795e-03 1.86742708e-01]
 [7.21358632e-04 8.08087212e-04 2.60515545e-02 ... 1.31145520e-03
  1.21500717e-03 1.86722887e-01]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 9
Total Number of Training instances: 5000


[ 41.99  24.77 236.43 ...  49.01   8.07  34.69]


Type y: <class 'nump

In [23]:
number_of_features = len(X[1])

#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=9, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.2))

    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [24]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 1000, 
                                 batch_size = 5)

In [25]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 4000 samples
Epoch 1/1000
4000/4000 [==============================] - 2s 553us/sample - loss: 69.4273 - mae: 69.4274 - mse: 21921.2832
Epoch 2/1000
4000/4000 [==============================] - 2s 391us/sample - loss: 58.4457 - mae: 58.4457 - mse: 20290.1211
Epoch 3/1000
4000/4000 [==============================] - 2s 382us/sample - loss: 50.7846 - mae: 50.7846 - mse: 18916.8086
Epoch 4/1000
4000/4000 [==============================] - 2s 400us/sample - loss: 48.5097 - mae: 48.5098 - mse: 18370.5918
Epoch 5/1000
4000/4000 [==============================] - 1s 366us/sample - loss: 48.0112 - mae: 48.0112 - mse: 18052.4355
Epoch 6/1000
4000/4000 [==============================] - 2s 394us/sample - loss: 48.1378 - mae: 48.1378 - mse: 18126.3652
Epoch 7/1000
4000/4000 [==============================] - 2s 400us/sample - loss: 47.8988 - mae: 47.8988 - mse: 18023.7383
Epoch 8/1000
4000/4000 [==============================] - 2s 386us/sample - loss: 47.6378 - mae: 47.6378 - mse: 17926

Epoch 67/1000
4000/4000 [==============================] - 2s 450us/sample - loss: 47.2233 - mae: 47.2233 - mse: 17858.3965
Epoch 68/1000
4000/4000 [==============================] - 2s 612us/sample - loss: 46.9360 - mae: 46.9360 - mse: 17816.6367
Epoch 69/1000
4000/4000 [==============================] - 2s 418us/sample - loss: 46.8249 - mae: 46.8249 - mse: 17767.0137- loss: 38
Epoch 70/1000
4000/4000 [==============================] - 2s 392us/sample - loss: 47.0834 - mae: 47.0834 - mse: 17825.0020
Epoch 71/1000
4000/4000 [==============================] - 2s 401us/sample - loss: 47.2688 - mae: 47.2688 - mse: 17879.4434
Epoch 72/1000
4000/4000 [==============================] - 1s 361us/sample - loss: 47.0079 - mae: 47.0079 - mse: 17867.0156
Epoch 73/1000
4000/4000 [==============================] - 2s 394us/sample - loss: 46.9504 - mae: 46.9504 - mse: 17831.3809
Epoch 74/1000
4000/4000 [==============================] - 2s 380us/sample - loss: 46.8550 - mae: 46.8550 - mse: 17859.892

4000/4000 [==============================] - 2s 377us/sample - loss: 46.5928 - mae: 46.5928 - mse: 17747.1191
Epoch 133/1000
4000/4000 [==============================] - 2s 384us/sample - loss: 46.4171 - mae: 46.4171 - mse: 17740.1543
Epoch 134/1000
4000/4000 [==============================] - 2s 414us/sample - loss: 46.4721 - mae: 46.4721 - mse: 17707.5312
Epoch 135/1000
4000/4000 [==============================] - 1s 366us/sample - loss: 46.7926 - mae: 46.7926 - mse: 17764.5977
Epoch 136/1000
4000/4000 [==============================] - 1s 369us/sample - loss: 46.3732 - mae: 46.3732 - mse: 17783.4453
Epoch 137/1000
4000/4000 [==============================] - 2s 423us/sample - loss: 46.5038 - mae: 46.5038 - mse: 17819.3027
Epoch 138/1000
4000/4000 [==============================] - 2s 397us/sample - loss: 46.6076 - mae: 46.6076 - mse: 17775.9023
Epoch 139/1000
4000/4000 [==============================] - 2s 392us/sample - loss: 46.5406 - mae: 46.5406 - mse: 17786.6973
Epoch 140/1000


4000/4000 [==============================] - 2s 401us/sample - loss: 46.2264 - mae: 46.2264 - mse: 17649.9355
Epoch 198/1000
4000/4000 [==============================] - 1s 367us/sample - loss: 46.0253 - mae: 46.0253 - mse: 17678.3652
Epoch 199/1000
4000/4000 [==============================] - 2s 377us/sample - loss: 46.1727 - mae: 46.1727 - mse: 17656.4180
Epoch 200/1000
4000/4000 [==============================] - 2s 396us/sample - loss: 46.0451 - mae: 46.0451 - mse: 17695.9395
Epoch 201/1000
4000/4000 [==============================] - 2s 415us/sample - loss: 46.3079 - mae: 46.3079 - mse: 17682.0078
Epoch 202/1000
4000/4000 [==============================] - 2s 380us/sample - loss: 46.1309 - mae: 46.1309 - mse: 17692.4219
Epoch 203/1000
4000/4000 [==============================] - 1s 358us/sample - loss: 46.1707 - mae: 46.1707 - mse: 17640.7188
Epoch 204/1000
4000/4000 [==============================] - 2s 398us/sample - loss: 46.2112 - mae: 46.2112 - mse: 17712.5430
Epoch 205/1000


4000/4000 [==============================] - 2s 403us/sample - loss: 45.7301 - mae: 45.7300 - mse: 17598.6367
Epoch 263/1000
4000/4000 [==============================] - 1s 366us/sample - loss: 45.7405 - mae: 45.7405 - mse: 17552.8496
Epoch 264/1000
4000/4000 [==============================] - 2s 398us/sample - loss: 45.7929 - mae: 45.7929 - mse: 17605.5000
Epoch 265/1000
4000/4000 [==============================] - 2s 386us/sample - loss: 45.7223 - mae: 45.7223 - mse: 17595.7305
Epoch 266/1000
4000/4000 [==============================] - 1s 362us/sample - loss: 45.8592 - mae: 45.8592 - mse: 17621.0469
Epoch 267/1000
4000/4000 [==============================] - 2s 381us/sample - loss: 45.6711 - mae: 45.6711 - mse: 17586.1992
Epoch 268/1000
4000/4000 [==============================] - 2s 392us/sample - loss: 45.7134 - mae: 45.7134 - mse: 17567.0781
Epoch 269/1000
4000/4000 [==============================] - 2s 386us/sample - loss: 45.6639 - mae: 45.6639 - mse: 17612.7559
Epoch 270/1000


...

4000/4000 [==============================] - 2s 455us/sample - loss: 44.2524 - mae: 44.2524 - mse: 16666.5176
Epoch 715/1000
4000/4000 [==============================] - 2s 588us/sample - loss: 44.2294 - mae: 44.2294 - mse: 16604.7930
Epoch 716/1000
4000/4000 [==============================] - 2s 453us/sample - loss: 44.3084 - mae: 44.3084 - mse: 16569.1738- loss: 35.4
Epoch 717/1000
4000/4000 [==============================] - 2s 505us/sample - loss: 44.2738 - mae: 44.2738 - mse: 16595.6094
Epoch 718/1000
4000/4000 [==============================] - 2s 436us/sample - loss: 44.2309 - mae: 44.2309 - mse: 16748.6191
Epoch 719/1000
4000/4000 [==============================] - 2s 460us/sample - loss: 44.0177 - mae: 44.0177 - mse: 16529.9219
Epoch 720/1000
4000/4000 [==============================] - 2s 464us/sample - loss: 43.9498 - mae: 43.9498 - mse: 16601.9805
Epoch 721/1000
4000/4000 [==============================] - 2s 464us/sample - loss: 44.0421 - mae: 44.0421 - mse: 16638.6641
Epo

4000/4000 [==============================] - ETA: 0s - loss: 43.8555 - mae: 43.8555 - mse: 16561.687 - 2s 499us/sample - loss: 43.7220 - mae: 43.7220 - mse: 16406.2891
Epoch 780/1000
4000/4000 [==============================] - 2s 439us/sample - loss: 43.7531 - mae: 43.7531 - mse: 16523.3867
Epoch 781/1000
4000/4000 [==============================] - 2s 412us/sample - loss: 43.9036 - mae: 43.9036 - mse: 16495.6758
Epoch 782/1000
4000/4000 [==============================] - 1s 374us/sample - loss: 43.8003 - mae: 43.8003 - mse: 16393.7891
Epoch 783/1000
4000/4000 [==============================] - 2s 383us/sample - loss: 43.8367 - mae: 43.8366 - mse: 16475.8691
Epoch 784/1000
4000/4000 [==============================] - 2s 490us/sample - loss: 43.8891 - mae: 43.8891 - mse: 16479.9902
Epoch 785/1000
4000/4000 [==============================] - 2s 376us/sample - loss: 43.6631 - mae: 43.6631 - mse: 16361.6699
Epoch 786/1000
4000/4000 [==============================] - 1s 371us/sample - loss

4000/4000 [==============================] - 2s 380us/sample - loss: 43.5117 - mae: 43.5117 - mse: 16214.9541
Epoch 844/1000
4000/4000 [==============================] - 1s 347us/sample - loss: 43.7009 - mae: 43.7009 - mse: 16356.6475
Epoch 845/1000
4000/4000 [==============================] - 2s 383us/sample - loss: 43.8203 - mae: 43.8203 - mse: 16373.9932
Epoch 846/1000
4000/4000 [==============================] - 1s 365us/sample - loss: 43.4595 - mae: 43.4595 - mse: 16299.6338
Epoch 847/1000
4000/4000 [==============================] - 1s 358us/sample - loss: 43.5161 - mae: 43.5161 - mse: 16282.8984
Epoch 848/1000
4000/4000 [==============================] - 1s 364us/sample - loss: 43.5548 - mae: 43.5548 - mse: 16313.5420
Epoch 849/1000
4000/4000 [==============================] - 1s 368us/sample - loss: 43.5436 - mae: 43.5436 - mse: 16289.6992
Epoch 850/1000
4000/4000 [==============================] - 2s 390us/sample - loss: 43.7565 - mae: 43.7565 - mse: 16304.3623
Epoch 851/1000


4000/4000 [==============================] - 2s 401us/sample - loss: 43.4223 - mae: 43.4223 - mse: 16201.7422
Epoch 909/1000
4000/4000 [==============================] - 2s 419us/sample - loss: 43.2170 - mae: 43.2171 - mse: 16182.3369
Epoch 910/1000
4000/4000 [==============================] - 2s 449us/sample - loss: 43.3732 - mae: 43.3732 - mse: 16123.4678
Epoch 911/1000
4000/4000 [==============================] - 2s 489us/sample - loss: 43.4482 - mae: 43.4482 - mse: 16208.2910
Epoch 912/1000
4000/4000 [==============================] - 2s 406us/sample - loss: 43.3571 - mae: 43.3571 - mse: 16169.2471
Epoch 913/1000
4000/4000 [==============================] - 2s 392us/sample - loss: 43.2873 - mae: 43.2873 - mse: 16186.9854
Epoch 914/1000
4000/4000 [==============================] - 1s 352us/sample - loss: 43.0674 - mae: 43.0674 - mse: 16160.2861
Epoch 915/1000
4000/4000 [==============================] - 2s 389us/sample - loss: 43.6390 - mae: 43.6390 - mse: 16272.7012
Epoch 916/1000


4000/4000 [==============================] - 2s 458us/sample - loss: 43.1554 - mae: 43.1553 - mse: 16030.5527
Epoch 974/1000
4000/4000 [==============================] - 2s 421us/sample - loss: 42.9502 - mae: 42.9502 - mse: 15916.0605
Epoch 975/1000
4000/4000 [==============================] - 2s 384us/sample - loss: 43.0489 - mae: 43.0489 - mse: 16058.8096
Epoch 976/1000
4000/4000 [==============================] - 2s 496us/sample - loss: 43.2370 - mae: 43.2370 - mse: 16066.0439
Epoch 977/1000
4000/4000 [==============================] - 2s 466us/sample - loss: 43.2999 - mae: 43.2999 - mse: 16061.6016
Epoch 978/1000
4000/4000 [==============================] - 2s 386us/sample - loss: 43.2025 - mae: 43.2025 - mse: 16011.8008
Epoch 979/1000
4000/4000 [==============================] - 2s 464us/sample - loss: 43.1686 - mae: 43.1686 - mse: 16177.9004
Epoch 980/1000
4000/4000 [==============================] - 2s 407us/sample - loss: 43.1039 - mae: 43.1039 - mse: 16000.3457
Epoch 981/1000


In [26]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-50.24005403 -42.60464417 -44.36739549 -39.60163519 -41.50389658]
Mean MAE: -43.663525091648104
Standard Deviation of the mean: 3.63401541025534
